Goal: Usind data about strokes, build a predictor of which conditions are most likely to result in a sroke.

In [1]:
from config import CONFIG
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import classes.BMICalculator as bmic
import classes.Encoders as enc
import pickle


### EDA

In [2]:
# EDA
data_loca= CONFIG["DATA_FILEPATH"]
child_bmi_data = CONFIG["CHILD_BMI"]
data = pd.read_csv(data_loca)
child_bmi_df = pd.read_csv(child_bmi_data)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
data["ever_married"].unique()

array(['Yes', 'No'], dtype=object)

In [4]:
len(data)

5110

In [5]:
data_df = data.dropna()
len(data_df)

4909

In [6]:
# round ages to nearest int
data_df["age"] = data_df["age"].round()

/var/folders/18/918s5vr16kl9091zgnswr5gm0000gn/T/ipykernel_15774/4176842324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["age"] = data_df["age"].round()


In [7]:
# can't calc bmi for ages below 2
data_df = data_df[data_df["age"] > 1]

In [8]:
len(data_df)

4828

In [ ]:
# # get histograms of all data to see spread
# import matplotlib.pyplot as plt
# cols = ["gender", "age", "hypertension", "heart_disease","ever_married", "work_type","Residence_type", "avg_glucose_level","bmi","smoking_status","stroke"]
# for col in cols:
#     # counts = data_df[col].value_counts().plot.barh()
#     # fig, ax = plt.subplots()
#     # ax.set_title(f"{col}")
#     # plt.show()

#     categories = data_df[col].value_counts().index
#     counts = data_df[col].value_counts().values
#     plt.barh(categories, counts)
#     plt.title(col)
#     plt.show()


In [ ]:
#  look at correlations
# import numpy as np
# import pandas as pd
# from scipy.stats import chi2_contingency

# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
#     r_corr = r - ((r - 1) ** 2) / (n - 1)
#     k_corr = k - ((k - 1) ** 2) / (n - 1)
#     return np.sqrt(phi2corr / min((k_corr - 1), (r_corr - 1)))

# def plot_categorical_correlation(features):
#     num_features = features.shape[1]
#     correlation_matrix = np.zeros((num_features, num_features))

#     for i in range(num_features):
#         for j in range(num_features):
#             correlation_matrix[i, j] = cramers_v(features[:, i], features[:, j])

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=0, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(num_features)]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(num_features):
#         for j in range(num_features):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Categorical Correlation Matrix (Cramer's V)")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()


# plot_categorical_correlation(data_df)


In [ ]:

# def plot_correlation_matrix(features):
#     # Calculate the correlation matrix using numpy.corrcoef
#     correlation_matrix = np.corrcoef(features, rowvar=False)

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(features.shape[1])]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(len(feature_names)):
#         for j in range(len(feature_names)):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Pearson's Correlation Matrix")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()

# plot_correlation_matrix(data_df)


### BMI conversion data

These files contain the L, M, and S parameters needed to generate exact percentiles and z-scores along with the percentile values for the 3rd, 5th, 10th, 25th, 50th, 75th, 90th, 95th, and 97th percentiles by sex (1=male; 2=female) and single month of age. The smoothed 85th percentile values are included in the BMI-for-age and weight-for-stature tables. Age is listed at the half month point for the entire month; for example, 1.5 months represents 1.0-1.99 months or 1.0 month up to but not including 2.0 months of age. The only exception is birth, which represents the point at birth. To obtain L, M, and S values at finer age or length/stature intervals interpolation could be used.

The LMS parameters are the median (M), the generalized coefficient of variation (S), and the power in the Box-Cox transformation (L). To obtain the value (X) of a given physical measurement at a particular z-score or percentile, use the following equation:

X = M (1 + LSZ)**(1/L), L ≠ 0

Or

X = M exp(SZ), L = 0

where the L, M, and S are the values from the appropriate table corresponding to the age in months of the child (** indicates an exponent, such that M(1+LSZ)**(1/L) means raising (1+LSZ) to the (1/L)th power and then multiplying the M; exp(X) is the exponentiation function, e to the power X). Z is the z-score that corresponds to the percentile. z-scores correspond exactly to percentiles, e.g., z-scores of -1.881, -1.645, -1.282, -0.674, 0, 0.674, 1.036, 1.282, 1.645, and 1.881 correspond to the 3rd, 5th, 10th, 25th, 50th, 75th, 85th, 90th, 95th, and 97th percentiles, respectively.

For example, to obtain the 5th percentile of weight-for-age for a 9-month-old male, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. For the 5th percentile, we would use Z=-1.645. Using the equation above, we calculate that the 5th percentile is 7.90 kg.

To obtain the z-score (Z) and corresponding percentile for a given measurement (X), use the following equation:

((X/M)**L) – 1
Z = ————————-, L≠0
LS

or

Z = ln(X/M)/S ,L=0

where X is the physical measurement (e.g. weight, length, head circumference, stature or calculated BMI value) and L, M and S are the values from the appropriate table corresponding to the age in months of the child (or length/stature). (X/M)**L means raising the quantity (X/M) to the Lth power.

For example, to obtain the weight-for-age z-score of a 9-month-old male who weighs 9.7 kg, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. Using the equation above, we calculate that the z-score for this child is 0.207. This z-score corresponds to the 58th percentile.

Z-scores and corresponding percentiles can be obtained from standard normal distribution tables found in statistics text books. Standard normal tables can also be found on the internet by doing a search on a “standard normal table.” In addition, many computer programs have pre-existing functions that convert Z-scores to percentiles and vice versa.



In [9]:
# chile BMI conversion df
child_bmi_df.head()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
1,1,24.5,-1.982373595,16.54777487,0.080127429,14.50347667,14.71929257,15.07117474,15.71962876,16.54777487,17.52129279,18.11954923,18.56110634,19.27889813,19.79194014
2,1,25.5,-1.924100169,16.49442763,0.079233994,14.46882381,14.68360841,15.03335725,15.67634464,16.49442763,17.45135039,18.03668013,18.46729593,19.16465965,19.66102345
3,1,26.5,-1.86549793,16.44259552,0.078389356,14.43459737,14.64843329,14.99619505,15.634035,16.44259552,17.38383656,17.95700228,18.37736191,19.05567423,19.5365754
4,1,27.5,-1.807261899,16.3922434,0.077593501,14.40082828,14.61378626,14.95969047,15.5926798,16.3922434,17.3187102,17.88047101,18.29125345,18.9518675,19.41848805


In [10]:
child_bmi_df.describe()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
count,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439
unique,3,220,439,439,439,439,439,439,439,439,439,439,439,439,439
top,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
freq,219,2,1,1,1,1,1,1,1,1,1,1,1,1,1


In [12]:
# # drop secondary heading (from joint male and female bmi chart)
child_bmi_df = child_bmi_df.drop(index = 219)

In [13]:
child_bmi_df["Sex"].unique()

array(['1', '2'], dtype=object)

In [14]:
child_bmi_df.dtypes

Sex       object
Agemos    object
L         object
M         object
S         object
P3        object
P5        object
P10       object
P25       object
P50       object
P75       object
P85       object
P90       object
P95       object
P97       object
dtype: object

In [15]:
child_bmi_df = child_bmi_df.astype(float)
child_bmi_df["Sex"] = child_bmi_df["Sex"].astype(int)

In [16]:
child_bmi_df.dtypes

Sex         int64
Agemos    float64
L         float64
M         float64
S         float64
P3        float64
P5        float64
P10       float64
P25       float64
P50       float64
P75       float64
P85       float64
P90       float64
P95       float64
P97       float64
dtype: object

In [17]:
child_bmi_df

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24.0,-2.011181,16.575028,0.080592,14.520953,14.737319,15.090328,15.741642,16.575028,17.557188,18.162195,18.609481,19.338011,19.859858
1,1,24.5,-1.982374,16.547775,0.080127,14.503477,14.719293,15.071175,15.719629,16.547775,17.521293,18.119549,18.561106,19.278898,19.791940
2,1,25.5,-1.924100,16.494428,0.079234,14.468824,14.683608,15.033357,15.676345,16.494428,17.451350,18.036680,18.467296,19.164660,19.661023
3,1,26.5,-1.865498,16.442596,0.078389,14.434597,14.648433,14.996195,15.634035,16.442596,17.383837,17.957002,18.377362,19.055674,19.536575
4,1,27.5,-1.807262,16.392243,0.077594,14.400828,14.613786,14.959690,15.592680,16.392243,17.318710,17.880471,18.291253,18.951867,19.418488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,2,237.5,-2.353501,21.689489,0.151755,17.435152,17.822557,18.480142,19.788872,21.689489,24.384258,26.397231,28.130338,31.596640,34.818095
435,2,238.5,-2.350529,21.701083,0.152221,17.434275,17.822591,18.481819,19.794202,21.701083,24.406856,26.429839,28.172905,31.663237,34.912504
436,2,239.5,-2.346962,21.711892,0.152716,17.431988,17.821267,18.482230,19.798457,21.711892,24.429104,26.462429,28.215745,31.730686,35.008306
437,2,240.0,-2.344958,21.716999,0.152975,17.430311,17.820090,18.481956,19.800176,21.716999,24.440096,26.478720,28.237271,31.764743,35.056751


## Encode data for training

In [27]:
def get_train_df(df, drop_cols, label_col):
    """
    Input: 
        df with encoded training data
        [drop_cols]; columns names to drop from df
        "label_col"; name of label column
    Output: 
        training data df, label df
    """
    labels = df[label_col]
    df = df.drop(label_col, axis = 1)
    train_df = df.drop(drop_cols, axis = 1)

    return train_df, labels

In [19]:
# split data by age 
child_df = data_df[data_df["age"] <= 19]
adult_df = data_df[data_df["age"] > 19]

In [20]:
# encode bmi for each df
enc_child_bmi =  enc.ChildBMIEncoder(bmi_table= child_bmi_df, data_table= child_df).encode()
enc_adult_bmi = enc.AdultBMIEncoder(adult_df).encode()

/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/classes/Encoders.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_table['bmi_buckets'] = bmi_buckets
/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/classes/Encoders.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_table['bmi_buckets'] = bmi_buckets


In [21]:
if len(enc_child_bmi.columns) == len(enc_adult_bmi.columns):
    new_df = pd.concat([enc_child_bmi,enc_adult_bmi])

else:
    print("Columns must match to concatenate dataframes")

new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_buckets
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,Unknown,1,obese
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,Unknown,0,over weight
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,Unknown,0,healthy weight
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,Unknown,0,healthy weight
282,33759,Female,3.0,0,0,No,children,Urban,73.74,16.0,Unknown,0,healthy weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,never smoked,0,healthy weight
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0,obese
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0,obese
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0,over weight


In [22]:
#one hot encode BMI
enc.one_hot_enc_bmi(new_df,"bmi_buckets")

In [23]:
new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_buckets,bmi_under,bmi_healthy,bmi_over,bmi_obese
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,Unknown,1,obese,0,0,0,1
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,Unknown,0,over weight,0,0,1,0
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,Unknown,0,healthy weight,0,1,0,0
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,Unknown,0,healthy weight,0,1,0,0
282,33759,Female,3.0,0,0,No,children,Urban,73.74,16.0,Unknown,0,healthy weight,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,never smoked,0,healthy weight,0,1,0,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0,obese,0,0,0,1
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0,obese,0,0,0,1
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0,over weight,0,0,1,0


In [25]:
age_enc = enc.AgeEncoder(new_df,"age")
new_df = age_enc.encode()
gend_enc = enc.GenderEncoder(new_df,"gender").encode()
marr_enc = enc.MarriageEncoder(new_df,"ever_married").encode()
res_enc = enc.ResidenceEncoder(new_df,"Residence_type").encode()
smoke_enc = enc.SmokingStatusEncoder(new_df, "smoking_status").encode()

new_df = smoke_enc
new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,a60,a70,a80,is_male,c_ever_married,live_urban,ss_1,ss_2,ss_3,ss_4
245,49669,Female,14.0,0,0,No,children,Rural,57.93,30.9,...,0,0,0,0,0,0,0,0,0,1
249,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,...,0,0,0,1,0,0,0,0,0,1
251,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,...,0,0,0,0,0,1,0,0,0,1
253,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,...,0,0,0,1,0,0,0,0,0,1
282,33759,Female,3.0,0,0,No,children,Urban,73.74,16.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,45010,Female,57.0,0,0,Yes,Private,Rural,77.93,21.7,...,0,0,0,0,1,0,0,1,0,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,...,0,0,1,0,1,1,0,1,0,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,...,0,0,0,0,1,0,0,1,0,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,...,0,0,0,1,1,0,0,0,1,0


In [28]:
# get train df, remove unecceasry columns
# removed avg glucose level for now (for demo)
# drop_cols = ["id", "gender", "age", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level"]
drop_cols = ["id", "gender", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level", "a0", "a10", "a20", "a30", "a40", "a50", "a60", "a70", "a80"]
train_df, labels = get_train_df(new_df, drop_cols, "stroke")
train_df

,age,hypertension,heart_disease,bmi_under,bmi_healthy,bmi_over,bmi_obese,is_male,c_ever_married,live_urban,ss_1,ss_2,ss_3,ss_4
245,14.0,0,0,0,0,0,1,0,0,0,0,0,0,1
249,3.0,0,0,0,0,1,0,1,0,0,0,0,0,1
251,8.0,0,0,0,1,0,0,0,0,1,0,0,0,1
253,14.0,0,0,0,1,0,0,1,0,0,0,0,0,1
282,3.0,0,0,0,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,57.0,0,0,0,1,0,0,0,1,0,0,1,0,0
5106,81.0,0,0,0,0,0,1,0,1,1,0,1,0,0
5107,35.0,0,0,0,0,0,1,0,1,0,0,1,0,0
5108,51.0,0,0,0,0,1,0,1,1,0,0,0,1,0


In [29]:
train_df.columns

Index(['age', 'hypertension', 'heart_disease', 'bmi_under', 'bmi_healthy',
       'bmi_over', 'bmi_obese', 'is_male', 'c_ever_married', 'live_urban',
       'ss_1', 'ss_2', 'ss_3', 'ss_4'],
      dtype='object')

In [30]:
labels

245     1
249     0
251     0
253     0
282     0
       ..
5102    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 4828, dtype: int64

### Train model

In [31]:
# split into testing and training data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, labels, test_size = 0.2, random_state = 42)

In [32]:
X_train.dtypes

age               float64
hypertension        int64
heart_disease       int64
bmi_under           int64
bmi_healthy         int64
bmi_over            int64
bmi_obese           int64
is_male             int64
c_ever_married      int64
live_urban          int64
ss_1                int64
ss_2                int64
ss_3                int64
ss_4                int64
dtype: object

In [33]:
# use SVM for classification
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [34]:
predictions = classifier.predict(X_test)

In [35]:
classifier.score(X_test,y_test)

0.9616977225672878

In [36]:
classifier.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [37]:
pickled_model = pickle.dumps(classifier)
with open ("model.txt", "wb") as file:
    file.write(pickled_model)

In [38]:
# import pickle
# with open ("model.txt", "rb") as file:
#     unpickled = pickle.load(file)
#     file.close()

In [39]:
# more metrics
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [40]:

classification_report(y_test,predictions)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

'              precision    recall  f1-score   support\n\n           0       0.96      1.00      0.98       929\n           1       0.00      0.00      0.00        37\n\n    accuracy                           0.96       966\n   macro avg       0.48      0.50      0.49       966\nweighted avg       0.92      0.96      0.94       966\n'